## Few-shots

In [ ]:

i=0
# Mock function to simulate API calls
def promptgroq_fewshots(csv, start_idx, chunksize, model, num_samples, initial_examples):
    global appliances
    answers = {'timestamps': [], 'app_predictions': {}}
    for app in appliances:
        answers['app_predictions'][app] = []
    attempts = 40
    predictions = []
    global api_keyover, client, i

    initial_examples_str = ", ".join(
    [f"Timestamp: {row['timestamp']}, Total Power: {row['total_power']}, Fridge Power: {row['fridge']},Dish washer Power: {row['dish washer']}"
     for idx, row in initial_examples.iterrows()]
)


    def get_prediction(run, csv=csv, start_idx=start_idx, chunksize=chunksize, model=model):
        global i, api_keyover, client
        while i < attempts:
            try:
                completion = client.chat.completions.create(
                    model='llama3-8b-8192',
                    messages=[
                        
                        {
                            "role": "system",
                            "content": """You are a helpful assistant that performs time series predictions for NILM (Non-Intrusive Load Monitoring). The user will provide an aggregate data sequence of power consumption and timestamps with a few examples of power consumed by fridge and dish washer, and you will predict the power consumption of the fridge corresponding to each timestamp in the data. The sequence is represented by decimal strings separated by commas for power values and HH:MM format for timestamps 
                                            DO NOT CHANGE THE TIMESTAMPS KEEP THEM SAME AS IN THE DATA. USE EXAMPLE STRING PROVIDED AS A TRAINING SET
                                            Also keep in mind the other appliances while predicting.
                                            Appliance Information:
                                            Low power consuming appliances - Lighting, Electronics, Fridge
                                            Moderate power consuming appliances - Bathroom GFI, Dishwasher, Microwave, Kitchen Outlets
                                            High power consuming appliances - Washer Dryer   
                                        """
                        },
                        {
                            "role": "user",
                            "content": f"""Predict the fridge and dish washer power consumption for the given time series data for the next {chunksize} timestamps by using the examples provided. Each timestamp in the input should have a corresponding output. Return your reply in the following format:
                                            [timestamp 0, prediction 0, timestamp 1, prediction 1, ..., timestamp {chunksize - 1}, prediction {chunksize - 1}] - the total length should be exactly {chunksize * 2}, starting from timestamp {start_idx} until the timestamp {start_idx + chunksize - 1}. Do not include any additional text or explanations.
                                            
                                            ### Initial Examples
                                            {initial_examples_str}
                                            
                                            ### Time Series Data
                                            {csv}
                                            """
                        }
                    ],
                    temperature=1,
                    top_p=1,
                    stream=False,
                    stop=None,
                )
            except Exception as e:
                print('API key limit reached', e)
                api_keyover += 1
                client = Groq(api_key=api_keys[api_keyover % 4])
                continue
            try:
                    answer = completion.choices[0].message.content.strip()
                    predictions_list = [item.strip() for item in answer.split(',') if item.strip()]

                    pred_dict_app = {}
                    
                    pred_timestamps = predictions_list[0::len(appliances)+1]
                    for i, app in enumerate(appliances):
                        pred_dict_app[app] = predictions_list[i+1::len(appliances)+1]

                    pred_cleaned = {} 
                    for app in appliances:
                        pred_values_cleaned = []
                        for value in pred_dict_app[app]:
                            try:
                                pred_values_cleaned.append(float(value))
                            except ValueError:
                                float_part = ''.join([char for char in value if char.isdigit() or char == '.' or char == '-'])
                                pred_values_cleaned.append(float(float_part))
                        pred_cleaned[app] = pred_values_cleaned
                    
                    pred_df = pd.DataFrame({
                        'timestamp': [ts.strip() for ts in pred_timestamps]
                    })

                    for appliance, predictions in pred_cleaned.items():
                        pred_df[appliance] = predictions

                    if len(pred_timestamps) == chunksize:
                        return pred_df
                    else:
                        i += 1
            except Exception as e:
                i += 1
                continue

        return None

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_run = {executor.submit(get_prediction, run): run for run in range(num_samples)}
        for future in tqdm(concurrent.futures.as_completed(future_to_run)):
            result = future.result()
            if result is not None:
                predictions.append(result)

    if len(predictions) < num_samples:
        return promptgroq(csv, start_idx, chunksize, model, num_samples)

    for i in range(chunksize):
        for app in appliances:
            sum = 0
            for j in range(num_samples):
                sum += predictions[j][app][i]
            answers['app_predictions'][app].append(sum / num_samples)
        answers['timestamps'].append(predictions[0]['timestamp'][i])
        
    return answers




In [ ]:
hours = 3
train_df = result_df[:hours*60]
test_df = result_df[hours*60:]
pred_df_list = []

# Select a small set of initial examples (e.g., first 5 rows)
initial_examples = train_df[['timestamp', 'total_power', 'fridge_power']]

In [ ]:
combinationstorun=[[0,15]]
model = 'llama3-70b-8192'
pred_df_list = []
# Run the loop
for start_idx, chunksize in combinationstorun:
    for start in tqdm(range(0, len(result_df) - (start_idx + chunksize), 15)):
        current_chunk = result_df.iloc[start:start+start_idx+chunksize]
        input_string = current_chunk[['timestamp', 'total_power']].to_csv(index=False, header=False)
        input_string = input_string.replace(" ", "").replace('\n', ',').replace(',', ' , ')
        answer = promptgroq(input_string, start + start_idx, chunksize, model, num_samples=2)
        predictions = answer['app_predictions']
        timestamps = answer['timestamps']
        predicted_df = pd.DataFrame({'Timestamp': timestamps})
        for appliance, predictions in predictions.items():
            predicted_df[appliance] = predictions
        print(predicted_df)
        pred_df_list.append(predicted_df)